RSA (Rivest-Shamir-Adleman) - это криптографический алгоритм, используемый для шифрования и расшифрования данных. Он основан на использовании открытого и закрытого ключей.

Вот как работает алгоритм RSA:

1. **Генерация ключей**:
   - Выбираются два различных простых числа $ p $ и $ q $.
   - Вычисляется их произведение $ n = p \times q $, которое будет использоваться в качестве модуля.
   - Вычисляется значение функции Эйлера от $ n $, обозначаемой как $ \phi(n) $. Для простых чисел $ p $ и $ q $ она равна $ \phi(n) = (p-1) \times (q-1) $.
   - Выбирается целое число $ e $, такое что $ 1 < e < \phi(n) $ и $ e $ является взаимно простым с $ \phi(n) $. $ e $ становится открытым ключом.
   - Находится целое число $ d $, такое что $ d \times e \equiv 1 \pmod{\phi(n)} $. $ d $ становится закрытым ключом.

2. **Шифрование сообщения**:
   - Сообщение $ M $ преобразуется в целое число $ m $, где $ 0 \leq m < n $.
   - Вычисляется шифротекст $ c $ по формуле: $ c \equiv m^e \pmod{n} $.

3. **Расшифрование сообщения**:
   - Шифротекст $ c $ преобразуется в исходное сообщение $ M $ по формуле: $ M \equiv c^d \pmod{n} $.


Это базовая реализация алгоритма RSA на Python. Для безопасности RSA обычно используются значительно большие простые числа (обычно длиной 1024 или 2048 бит), чем в этом примере.

In [3]:
import random
import math


def is_prime(n, k=5):
    """Check if a number is prime using Miller-Rabin primality test."""
    if n <= 1:
        return False
    if n <= 3:
        return True
    if n % 2 == 0:
        return False

    def check(a, s, d, n):
        x = pow(a, d, n)
        if x == 1:
            return True
        for _ in range(s - 1):
            if x == n - 1:
                return True
            x = pow(x, 2, n)
        return x == n - 1

    s, d = 0, n - 1
    while d % 2 == 0:
        d //= 2
        s += 1

    for _ in range(k):
        a = random.randrange(2, n - 1)
        if not check(a, s, d, n):
            return False
    return True


def gcd(a, b):
    """Calculate the greatest common divisor of two numbers."""
    while b != 0:
        a, b = b, a % b
    return a


def mod_inverse(a, m):
    """Calculate the modular inverse of a number."""
    m0, x0, x1 = m, 0, 1
    while a > 1:
        q = a // m
        m, a = a % m, m
        x0, x1 = x1 - q * x0, x0
    return x1 + m0 if x1 < 0 else x1


def generate_keypair(bit_length=1024):
    """Generate a RSA key pair."""
    while True:
        p = random.randint(2 ** (bit_length - 1), 2 ** bit_length - 1)
        if is_prime(p):
            break
    while True:
        q = random.randint(2 ** (bit_length - 1), 2 ** bit_length - 1)
        if is_prime(q) and q != p:
            break
    n = p * q
    phi = (p - 1) * (q - 1)
    e = 65537  # Common value for e (prime)
    d = mod_inverse(e, phi)
    return (e, n), (d, n)


def encrypt(message, public_key):
    """Encrypt a message using RSA."""
    e, n = public_key
    encrypted_msg = [pow(ord(char), e, n) for char in message]
    return encrypted_msg


def decrypt(encrypted_message, private_key):
    """Decrypt an encrypted message using RSA."""
    d, n = private_key
    decrypted_msg = [chr(pow(char, d, n)) for char in encrypted_message]
    return ''.join(decrypted_msg)


def save_private_key(private_key, filename):
    """Save the private key to a file."""
    with open(filename, 'w') as f:
        f.write(','.join(map(str, private_key)))


def load_private_key(filename):
    """Load the private key from a file."""
    with open(filename, 'r') as f:
        d, n = map(int, f.readline().split(','))
    return d, n


# Generate key pair
public_key, private_key = generate_keypair()
print("Public key:", public_key)

# Save private key to file
save_private_key(private_key, "private_key.txt")

# Encrypt a message
message = "This is a secret message. Do not share it with anyone!"
encrypted_message = encrypt(message, public_key)
print("Encrypted message:", encrypted_message)

# Load private key from file
loaded_private_key = load_private_key("private_key.txt")

# Decrypt the encrypted message
decrypted_message = decrypt(encrypted_message, loaded_private_key)
print("Decrypted message:", decrypted_message)


Public key: (65537, 17569922231483491457748187672546654130257115931532532658991749142344798221192593170285744964640146846815081668163110924378193572925512540505511135707476346362736611989095905719052925580959812469498329592574231925076807567982650093432864802497210026705846904799565811385360999466997392601227544671352663480371483317331312014245225500986728340636514802351128321719984515635936675426467661080990345937971652906642409755052624596735224530597626412378554517247087280797559757609381134880923747137919505577682081558589921849020646994582489812824408259868220658798803753006549591840270343792352051772944488405022597509929599)
Encrypted message: [...]
Decrypted message: This is a secret message. Do not share it with anyone!
